In [3]:
# DIRECTORY SET
import os
import sys
from pathlib import Path
base_dir=Path(os.getcwd()).parent
# os.chdir(os.path.join(base_dir, 'serverproject'))
os.chdir(base_dir)
print(os.getcwd())

# Load dotenv
import dotenv
dotenv.load_dotenv()

# DJANGO SETUP
import django
sys.path.append(os.path.abspath(''))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "serverproject.settings")
django.setup()

# Import async modules
import asyncio
from asgiref.sync import sync_to_async

# Import display modules
from IPython.display import display, Markdown

# Import other modules
import faiss

# import reloading
from importlib import reload

d:\DestinyFolder\DestinyRecaps\DestinyRecapsApi


In [4]:
times=[]
import time
t_topics=time.time()
times.append({"topics":t_topics})
t_annotations=time.time()
times.append({"annotations":t_annotations})

for i in range(1,len(times)):
    print(list(times[i].keys())[0], ": ", list(times[i].values())[0]-list(times[i-1].values())[0])

annotations :  0.0010106563568115234


In [5]:
%load_ext autoreload
%autoreload 2

from destinyapp.models import StreamRecapData, FastRecapData

from core import services
from core import utils
from core import controller

In [6]:
video_id="9KENE2wOphQ_no_recap"
cost=0

In [7]:
import json
from youtube_transcript_api import YouTubeTranscriptApi

if not os.path.exists("raw_transcript.json"):
# if True:
    raw_transcript=YouTubeTranscriptApi.get_transcript(video_id)
    with open("raw_transcript.json", "w") as f:
        json.dump(raw_transcript, f)
else:
    with open("raw_transcript.json", "r") as f:
        raw_transcript=json.load(f)

In [8]:
transcript, linked_transcript=services.fast.transcript_processing.process_yt_transcript(raw_transcript, video_id)

In [10]:
print(len(transcript))

313391


In [15]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
len(enc.encode(transcript))

70061

In [16]:
(120/70)*5.25

9.0

In [59]:
# summarize the transcript
transcript_summary_prompt=[{"role":"system", "content":"Your job is to write a very detailed summary of a transcript of a video, go through each major topic that is discussed"}, {"role":"user", "content":transcript}]
transcript_summary, temp_cost = await utils.async_response_handler(transcript_summary_prompt, utils.ModelNameEnum.llama_3_1_8B_instant)

In [46]:
print(transcript_summary)

The conversation involves a discussion among individuals on a video stream about various political and social issues, including the concept of court packing, the legitimacy of the Supreme Court, and the potential implications of President Trump's behavior on the Constitution.

The discussion begins with a description of the current situation of the Supreme Court, noting that it has been packed with conservative justices, which has led to a significant shift in the Court's ideology. The panelists discuss the potential consequences of this shift, including the ability of the Court to impose its will on the other branches of government.

Destiny, one of the panelists, argues that the Supreme Court is no longer functioning as a legitimate institution, and that the Republican Party has been using it to advance their own interests. He suggests that the answer to this problem is to pack the Court with more justices who will defend the rule of law and the Constitution.

Lact toid, another pane

In [44]:
print(len(transcript_summary))

11965


In [24]:
len(transcript)

313391

In [54]:
topics_str, major_topics, temp_cost=await services.fast.data_gen.create_topics(transcript_summary, model_name=utils.ModelNameEnum.gpt_4o_mini)

In [55]:
major_topics

["Supreme Court's current situation",
 'Court packing',
 'Legitimacy of the Supreme Court',
 'Political behavior and party influence',
 'Judicial interpretation and Originalism',
 'Rule of Law and Judicial Power']

In [74]:
text_chunks_no_overlap = await services.stream_plot.data_gen.create_text_chunks(transcript, 0)
text_chunk_batches = await services.stream_plot.data_gen.generate_text_chunk_batches(text_chunks_no_overlap)
# responses, annotated_results, temp_cost = await services.fast.data_gen.annotate_all_batches(text_chunk_batches, topics_str)
responses, annotated_results, temp_cost = await services.fast.data_gen.annotate_all_batches_no_recap(text_chunk_batches)

Cost:  0.00074235
Cost:  0.0010152
Cost:  0.0010605
Cost:  0.0010573499999999999
Cost:  0.0010267499999999999
Cost:  0.00103815
Cost:  0.00104865
Cost:  0.0010462499999999999
Cost:  0.00104535
Cost:  0.0011334
Cost:  0.0010782
Cost:  0.00108705
Cost:  0.00111675
Cost:  0.00113175
Cost:  0.00113715
Cost:  0.0011104499999999998
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  20
Results:  14


In [76]:
major_topics=set()
for ar in annotated_results:
    major_topics.add(ar["category"])
major_topics=list(major_topics)
print(len(major_topics))


253


In [94]:
annotated_segments, category_locations = await services.stream_plot.data_processing.create_segments(linked_transcript, annotated_results, major_topics, transcript)
# annotated_segments[-1]["category"]="non categorized"
plot_segments=await services.stream_plot.data_processing.annotated_to_plot_segments(annotated_segments)
plot_object=await services.stream_plot.data_processing.create_plot_object(plot_segments, category_locations, video_id)

252
Media Landscape Changes
Politics and Communication
Skills for Political Success
Tech Setup and Equipment Discussion
Champion Mindset Discussion
Podcast Setup Quality Discussion
Global Conflicts and Leadership
Public Speaking Challenges
Political Leadership Critique
Winning and Losing Mindset
Graphics Card Recommendations
Qualities of Successful Politicians
Political Debate Discussion
International Relations Discussion
Medical Marijuana and Political Conversation
Audio Quality Comparison
Geopolitical Analysis
Negotiation Strategies
Casual Chat and Tech Discussion
Voting Rights and Regulations
Geopolitical Strategy
Military Strategy Analysis
Graphics Card Power Consumption
Immigration Solutions
Election and Economic Discussion
Crime and Immigration
Graphics Card Specifications
Military and Foreign Policy
Communication in Politics
Technology and Politics Discussion
Social Media and Communication
Drug Use and Perception
Media and Political Relationships
Political Communication and Rela

In [95]:
plot_object, temp_cost=await services.stream_plot.extra_annotations.recap_segments(plot_object)

In [96]:
plot_object, temp_cost=await services.stream_plot.extra_annotations.recap_abstractions(plot_object)

In [97]:
data_grabbber=await sync_to_async(FastRecapData.objects.filter)(video_id=video_id)
fast_recap_data=await sync_to_async(data_grabbber.first)()
if not fast_recap_data:
    print("NOT FOUND")
    fast_recap_data=FastRecapData(video_id=video_id)

from dataclasses import asdict
fast_recap_data.plot_object=asdict(plot_object)
await sync_to_async(fast_recap_data.save)()

# full execution

In [ ]:
video_id="zKeMP1rnKUI"
cost=0

In [ ]:
# Make transcript
# if not os.path.exists("raw_transcript.json"):
#     raw_transcript=YouTubeTranscriptApi.get_transcript(video_id)
#     with open("raw_transcript.json", "w") as f:
#         json.dump(raw_transcript, f)
# else:
#     with open("raw_transcript.json", "r") as f:
#         raw_transcript=json.load(f)

import json
from youtube_transcript_api import YouTubeTranscriptApi
from dataclasses import asdict

raw_transcript=YouTubeTranscriptApi.get_transcript(video_id)
transcript, linked_transcript=services.fast.transcript_processing.process_yt_transcript(raw_transcript, video_id)


# Make Topics
topic_sys_prompt="""You will be given an entire transcript and your job is to come up with the categories discussed during the transcript.

Step 1: You will start by listing as many topic categories as you can in chronological order from the transcript. This may be 20 or so.

Step 2: Boil down the categories to the top 8-12 categories.

Step 3: Boil down to the top categories for which to make a content graph.


When formatting these steps, please use the following format:

Step 1:
1. Topic 1
2. Topic 2
...
20. Topic 20

Step 2:
1. Topic 1
2. Topic 2
...

Step 3:
1. Topic 1
2. Topic 2
...

"""
user_prompt="Here is the transcript: "+transcript

full_topic_gen_prompt=[{"role":"system", "content":topic_sys_prompt}, {"role":"user", "content":user_prompt}]
topics_response, temp_cost = await utils.async_response_handler(full_topic_gen_prompt, utils.ModelNameEnum.gpt_4o_mini)
cost+=temp_cost


# Process Topics
topics_str=topics_response.split("Step 3:")[-1]
major_topics_raw=topics_str.split("\n")
major_topics=[]
for t in major_topics_raw:
    t=t.strip()
    if (t=="") or not t[0].isdigit():
        continue
    else:
        major_topics.append(t[2:].strip())



# Annotate Text batches
text_chunks_no_overlap = await services.stream_plot.data_gen.create_text_chunks(transcript, 0)
text_chunk_batches = await services.stream_plot.data_gen.generate_text_chunk_batches(text_chunks_no_overlap)
responses, annotated_results, temp_cost = await services.fast.data_gen.annotate_all_batches(text_chunk_batches, topics_str)
cost+=temp_cost


# Create Plot
annotated_segments, category_locations = await services.stream_plot.data_processing.create_segments(linked_transcript, annotated_results, major_topics, transcript)
plot_segments=await services.stream_plot.data_processing.annotated_to_plot_segments(annotated_segments)
plot_object=await services.stream_plot.data_processing.create_plot_object(plot_segments, category_locations, video_id)

In [ ]:
len(user_prompt)

In [ ]:
# save plot
# get existing data if it exists
data_grabbber=await sync_to_async(StreamRecapData.objects.filter)(video_id=video_id)
stream_recap_data=await sync_to_async(data_grabbber.first)()
if not stream_recap_data:
    print("NOT FOUND")
    stream_recap_data=StreamRecapData(video_id=video_id)

from dataclasses import asdict
stream_recap_data.plot_object=asdict(plot_object)
await sync_to_async(stream_recap_data.save)()

In [ ]:
fast_recap_data=FastRecapData(video_id=video_id, plot_object=asdict(plot_object))

In [ ]:
serializer=utils.database_operations.FastRecapSerializer(data=fast_recap_data)

In [ ]:
serialized_data = await sync_to_async(lambda: utils.database_operations.FastRecapSerializer(fast_recap_data).data)()

In [ ]:
serialized_data

In [ ]:
await sync_to_async(serializer.is_valid)(raise_exception=True)

In [ ]:
# print why the asdict(plot_object) is failing to pass as json
json_test_data=asdict(plot_object)

def find_invalid_json(data):
    if isinstance(data, dict):
        for key, value in data.items():
            try:
                json.dumps({key: value})
            except json.JSONDecodeError:
                print(f"Invalid JSON at key: {key}")
                print(f"Value: {value}")
            find_invalid_json(value)
    elif isinstance(data, list):
        for item in data:
            find_invalid_json(item)

find_invalid_json(json_test_data)


In [ ]:
major_topics

In [ ]:
plot_object.abstractions

In [ ]:
str(asdict(plot_object))[4171:]

In [ ]:
print("http://localhost:5173/stream_plot?video_id="+video_id)

In [ ]:
# check if asdict(plot_object) is a valid json
json.dumps(asdict(plot_object))

In [ ]:
stream_recap_data.plot_object=asdict(plot_object)


In [ ]:
print(asdict(plot_object))